In [1]:
#pip install turicreate
!pip install -U turicreate

     |████████████████████████████████| 92.0MB 110kB/s 
     |████████████████████████████████| 86.4MB 48kB/s 
     |████████████████████████████████| 327kB 46.6MB/s 
     |████████████████████████████████| 3.4MB 38.2MB/s 
     |████████████████████████████████| 3.8MB 40.0MB/s 
     |████████████████████████████████| 450kB 52.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320848 sha256=d0c503bf6c54526f8090d53d4006cf20f0715ec164c405f89489f94309201062
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for prettytable: filename=prettytable-0.7.2-cp36-none-any.whl size=13700 sha256=1a01fef7595a83314e8b83f874e8616cf4ce114226ecd008b0a668044c637c5c
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=a

In [2]:
from os import path
import turicreate as tc
from datetime import datetime

In [3]:
import turicreate as tc

`<h1> Load Data </h1> MovieLens dataset collected by the GroupLens Research Project at the University of Minnesota. For more information, see http://grouplens.org/datasets/movielens/

In [4]:
#uploaded_rating = files.upload()
actions = tc.SFrame.read_csv('ratings.csv')

Finished parsing file /content/ratings.csv

Parsing completed. Parsed 100 lines in 1.63401 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2042753 lines. Lines per second: 763127

Finished parsing file /content/ratings.csv

Parsing completed. Parsed 8534968 lines in 5.81642 secs.

In [5]:
# Table of movies we are recommending: movieId, title, genres
items = tc.SFrame.read_csv('movies.csv')

Finished parsing file /content/movies.csv

Parsing completed. Parsed 100 lines in 0.088744 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/movies.csv

Parsing completed. Parsed 27278 lines in 0.046241 secs.

In [6]:
actions.head()

userId,movieId,rating,timestamp
1,2,3.5,1112486027
1,29,3.5,1112484676
1,32,3.5,1112484819
1,47,3.5,1112484727
1,50,3.5,1112484580
1,112,3.5,1094785740
1,151,4.0,1094785734
1,223,4.0,1112485573
1,253,4.0,1112484940
1,260,4.0,1112484826


In [7]:
items.head()

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy ...
2,Jumanji (1995),Adventure|Children|Fantasy ...
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride PartII (1995) ...,Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


<h1> Prepare Data </h1>

In [8]:
# Prepare the data by removing items that are rare
rare_items = actions.groupby('movieId', tc.aggregate.COUNT).sort('Count')
rare_items = rare_items[rare_items['Count'] <= 5]
items = items.filter_by(rare_items['movieId'], 'movieId', exclude=True)
actions = actions.filter_by(rare_items['movieId'], 'movieId', exclude=True)
actions['timestamp'] = actions['timestamp'].astype(datetime)

In [9]:
# Extract year, title, and genre
items['year'] = items['title'].apply(lambda x: x[-5:-1])
items['title'] = items['title'].apply(lambda x: x[:-7])
#items['genres'] = items['genres'].apply(lambda x: x.split('|'))

<h1> Train the Model </h1>

In [10]:
training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'userId', 'movieId')
#model = tc.recommender.create(training_data, 'userId', 'movieId')

In [11]:
model = tc.item_similarity_recommender.create(training_data, 'userId','movieId',target="rating",similarity_type='cosine')

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 8489725 observations with 58957 users and 14211 items.

Data prepared in: 8.56738s

...

| 1m 3s                               | 100              | 14211           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 65.0023s

<h1> Model Predict </h1>

In [12]:
results = model.recommend()

recommendations finished on 1000/58957 queries. users per second: 4401.47

recommendations finished on 2000/58957 queries. users per second: 4366.58

recommendations finished on 3000/58957 queries. users per second: 4483.5

recommendations finished on 4000/58957 queries. users per second: 4538.7

recommendations finished on 5000/58957 queries. users per second: 4548.55

...

recommendations finished on 54000/58957 queries. users per second: 4614.37

recommendations finished on 55000/58957 queries. users per second: 4617.48

recommendations finished on 56000/58957 queries. users per second: 4616.39

recommendations finished on 57000/58957 queries. users per second: 4616.4

recommendations finished on 58000/58957 queries. users per second: 4618.74

In [13]:
print(results[0:5])

+--------+---------+--------------------+------+
| userId | movieId |       score        | rank |
+--------+---------+--------------------+------+
|   1    |   2571  | 0.6964869751249041 |  1   |
|   1    |   1270  | 0.6578884959220886 |  2   |
|   1    |   1580  | 0.5610426310130529 |  3   |
|   1    |   2115  | 0.5410117881638663 |  4   |
|   1    |   1527  | 0.5375504503931318 |  5   |
+--------+---------+--------------------+------+
[5 rows x 4 columns]




<h1> Find Similar Items </h1>

In [14]:
similar_items = model.get_similar_items()

In [15]:
print(similar_items)

+---------+---------+---------------------+------+
| movieId | similar |        score        | rank |
+---------+---------+---------------------+------+
|    2    |   367   |  0.5152812004089355 |  1   |
|    2    |   500   |  0.5055213570594788 |  2   |
|    2    |   364   |  0.5045807361602783 |  3   |
|    2    |   586   | 0.49369752407073975 |  4   |
|    2    |   480   |  0.491912841796875  |  5   |
|    2    |   588   | 0.46647292375564575 |  6   |
|    2    |   377   | 0.45924896001815796 |  7   |
|    2    |   317   | 0.45645421743392944 |  8   |
|    2    |   208   |  0.4472547173500061 |  9   |
|    2    |   153   | 0.44424325227737427 |  10  |
+---------+---------+---------------------+------+
[142110 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


<h1> Popularity Based Model<h1>

In [17]:
pop_model = tc.popularity_recommender.create(training_data, 'userId','movieId',target="rating")

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 8489725 observations with 58957 users and 14211 items.

Data prepared in: 8.56526s

8489725 observations to process; with 14211 unique items.

<h1> Model Comparison <h1>

In [18]:
tc.recommender.util.compare_models(validation_data, [model, pop_model], model_names=["item2item", "popularity"])

PROGRESS: Evaluate model item2item


recommendations finished on 1000/1000 queries. users per second: 1798.07


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |  0.3619999999999999 | 0.02547648725652714 |
|   2    |        0.3285       | 0.04595736143781055 |
|   3    |  0.3156666666666663 | 0.06499186837248325 |
|   4    |        0.301        |  0.0788392797585694 |
|   5    | 0.28820000000000007 | 0.09193025569575684 |
|   6    |  0.2781666666666667 | 0.10368574283030746 |
|   7    | 0.26528571428571424 | 0.11308066816648558 |
|   8    | 0.25475000000000003 | 0.12146416860303044 |
|   9    |  0.2498888888888889 | 0.13262686270302135 |
|   10   | 0.24269999999999994 | 0.14088886689367552 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 3.4918197617743125

Per User RMSE (best)
+--------+--------------------+-------+
| userId |        rmse        | count |
+--------+----------

recommendations finished on 1000/1000 queries. users per second: 2210.02


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    |         0.0          |         0.0          |
|   2    | 0.03850000000000001  | 0.005649709389925732 |
|   3    | 0.02566666666666665  | 0.005649709389925734 |
|   4    | 0.019249999999999996 | 0.005649709389925734 |
|   5    | 0.015399999999999992 | 0.005649709389925734 |
|   6    | 0.012833333333333327 | 0.005649709389925734 |
|   7    | 0.011000000000000001 | 0.005649709389925734 |
|   8    | 0.013125000000000001 | 0.007051907800636157 |
|   9    | 0.01599999999999998  | 0.009316812257479054 |
|   10   | 0.016999999999999987 | 0.010372305437016524 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9315491453935739

Per User RMSE (best)
+--------+---------------------+-------+
| userId |         rmse        

[{'precision_recall_by_user': Columns:
  	userId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 18000
  
  Data:
  +--------+--------+-----------+--------+-------+
  | userId | cutoff | precision | recall | count |
  +--------+--------+-----------+--------+-------+
  | 29099  |   1    |    0.0    |  0.0   |   30  |
  | 29099  |   2    |    0.0    |  0.0   |   30  |
  | 29099  |   3    |    0.0    |  0.0   |   30  |
  | 29099  |   4    |    0.0    |  0.0   |   30  |
  | 29099  |   5    |    0.0    |  0.0   |   30  |
  | 29099  |   6    |    0.0    |  0.0   |   30  |
  | 29099  |   7    |    0.0    |  0.0   |   30  |
  | 29099  |   8    |    0.0    |  0.0   |   30  |
  | 29099  |   9    |    0.0    |  0.0   |   30  |
  | 29099  |   10   |    0.0    |  0.0   |   30  |
  +--------+--------+-----------+--------+-------+
  [18000 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more ro

<h1> Adding Item to Item similiarty by Pearson correlation <h1>

In [25]:
pearson_model = tc.item_similarity_recommender.create(training_data, 'userId','movieId',target="rating",similarity_type='pearson')

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 8489725 observations with 58957 users and 14211 items.

Data prepared in: 8.37343s

...

| 1m 21s                              | 100              | 14211           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 81.5902s

In [26]:
tc.recommender.util.compare_models(validation_data, [model, pop_model,pearson_model], model_names=["item2item", "popularity","pearson"])

PROGRESS: Evaluate model item2item


recommendations finished on 1000/1000 queries. users per second: 1894.49


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |        0.362        | 0.025476487256527142 |
|   2    | 0.32850000000000007 | 0.04595736143781055  |
|   3    |  0.3156666666666666 | 0.06499186837248316  |
|   4    |        0.301        | 0.07883927975856937  |
|   5    |  0.2881999999999999 | 0.09193025569575682  |
|   6    |  0.2781666666666665 | 0.10368574283030747  |
|   7    |  0.2652857142857143 | 0.11308066816648563  |
|   8    | 0.25474999999999987 | 0.12146416860303041  |
|   9    | 0.24988888888888885 | 0.13262686270302135  |
|   10   |  0.2427000000000001 |  0.1408888668936755  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 3.4918197617743125

Per User RMSE (best)
+--------+--------------------+-------+
| userId |        rmse        | count |
+-----

recommendations finished on 1000/1000 queries. users per second: 2156.89


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    |         0.0          |         0.0          |
|   2    | 0.03850000000000001  | 0.005649709389925732 |
|   3    | 0.02566666666666665  | 0.005649709389925734 |
|   4    | 0.019249999999999996 | 0.005649709389925734 |
|   5    | 0.015399999999999992 | 0.005649709389925734 |
|   6    | 0.012833333333333327 | 0.005649709389925734 |
|   7    | 0.011000000000000001 | 0.005649709389925734 |
|   8    | 0.013125000000000001 | 0.007051907800636157 |
|   9    | 0.01599999999999998  | 0.009316812257479054 |
|   10   | 0.016999999999999987 | 0.010372305437016524 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9315491453935739

Per User RMSE (best)
+--------+---------------------+-------+
| userId |         rmse        

recommendations finished on 1000/1000 queries. users per second: 1818.83


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    |         0.0          |          0.0          |
|   2    | 0.027000000000000003 | 0.0038532665672431302 |
|   3    | 0.024999999999999988 |  0.005448559964638377 |
|   4    | 0.018750000000000003 |  0.005448559964638377 |
|   5    | 0.015199999999999988 |  0.005615226631305046 |
|   6    | 0.012666666666666661 |  0.005615226631305046 |
|   7    | 0.012285714285714287 | 0.0066158821192588415 |
|   8    | 0.013875000000000004 |  0.007472877756606111 |
|   9    | 0.01566666666666664  |  0.009090935064496613 |
|   10   | 0.017700000000000007 |  0.010945895297755758 |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9287756236226004

Per User RMSE (best)
+--------+----------------------+-------+
| userId |      

[{'precision_recall_by_user': Columns:
  	userId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 18000
  
  Data:
  +--------+--------+-----------+--------+-------+
  | userId | cutoff | precision | recall | count |
  +--------+--------+-----------+--------+-------+
  | 29099  |   1    |    0.0    |  0.0   |   30  |
  | 29099  |   2    |    0.0    |  0.0   |   30  |
  | 29099  |   3    |    0.0    |  0.0   |   30  |
  | 29099  |   4    |    0.0    |  0.0   |   30  |
  | 29099  |   5    |    0.0    |  0.0   |   30  |
  | 29099  |   6    |    0.0    |  0.0   |   30  |
  | 29099  |   7    |    0.0    |  0.0   |   30  |
  | 29099  |   8    |    0.0    |  0.0   |   30  |
  | 29099  |   9    |    0.0    |  0.0   |   30  |
  | 29099  |   10   |    0.0    |  0.0   |   30  |
  +--------+--------+-----------+--------+-------+
  [18000 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more ro

<h1> Matrix Factorization <h1?

In [35]:
training_data_mf = training_data.remove_column('timestamp')

In [36]:
mf_model = tc.factorization_recommender.create(training_data_mf,'userId','movieId',target="rating")

Preparing data set.

Data has 8489725 observations with 58957 users and 14211 items.

Data prepared in: 8.67769s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.526693

Final training RMSE: 0.724822

In [37]:
validation_data_mf = validation_data.remove_column('timestamp')

In [39]:
tc.recommender.util.compare_models(validation_data_mf, [pearson_model,mf_model], model_names=["pearson","mf"])

PROGRESS: Evaluate model pearson


recommendations finished on 1000/1000 queries. users per second: 1783.5


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    |         0.0          |          0.0          |
|   2    | 0.027000000000000003 |  0.003853266567243131 |
|   3    | 0.02499999999999998  |  0.005448559964638377 |
|   4    | 0.018749999999999992 |  0.005448559964638377 |
|   5    | 0.015199999999999993 |  0.005615226631305042 |
|   6    | 0.01266666666666666  |  0.005615226631305042 |
|   7    | 0.012285714285714283 |  0.006615882119258838 |
|   8    |       0.013875       | 0.0074728777566061095 |
|   9    | 0.015666666666666634 |  0.00909093506449661  |
|   10   | 0.017700000000000007 |  0.010945895297755758 |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9287756236226004

Per User RMSE (best)
+--------+----------------------+-------+
| userId |      

recommendations finished on 1000/1000 queries. users per second: 1841.57


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |         0.002         | 1.6198571076619858e-05 |
|   2    |         0.003         | 9.625078637772793e-05  |
|   3    |  0.002999999999999999 | 0.00015040524553628883 |
|   4    |  0.003000000000000001 | 0.0002924727955866994  |
|   5    | 0.0031999999999999997 | 0.0003970558743077092  |
|   6    |  0.002999999999999999 | 0.0004400628673147021  |
|   7    |  0.002857142857142857 | 0.00046969938115195957 |
|   8    |        0.003375       | 0.0007926100824433812  |
|   9    |  0.003666666666666668 | 0.0008730467843142697  |
|   10   |  0.003700000000000002 |  0.00106105960482709   |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.8058449573035918

Per User RMSE (best)
+--------+--------------------

[{'precision_recall_by_user': Columns:
  	userId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 18000
  
  Data:
  +--------+--------+-----------+--------+-------+
  | userId | cutoff | precision | recall | count |
  +--------+--------+-----------+--------+-------+
  | 29099  |   1    |    0.0    |  0.0   |   30  |
  | 29099  |   2    |    0.0    |  0.0   |   30  |
  | 29099  |   3    |    0.0    |  0.0   |   30  |
  | 29099  |   4    |    0.0    |  0.0   |   30  |
  | 29099  |   5    |    0.0    |  0.0   |   30  |
  | 29099  |   6    |    0.0    |  0.0   |   30  |
  | 29099  |   7    |    0.0    |  0.0   |   30  |
  | 29099  |   8    |    0.0    |  0.0   |   30  |
  | 29099  |   9    |    0.0    |  0.0   |   30  |
  | 29099  |   10   |    0.0    |  0.0   |   30  |
  +--------+--------+-----------+--------+-------+
  [18000 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more ro

In [40]:
mf_rank_model = tc.ranking_factorization_recommender.create(training_data_mf,'userId','movieId',target="rating")

Preparing data set.

Data has 8489725 observations with 58957 users and 14211 items.

Data prepared in: 8.79002s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.762495

Final training RMSE: 0.712547

In [42]:
tc.recommender.util.compare_models(validation_data_mf, [model,mf_rank_model], model_names=["cosine","mf rank"])

PROGRESS: Evaluate model cosine


recommendations finished on 1000/1000 queries. users per second: 1770.36


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.3619999999999999 | 0.025476487256527135 |
|   2    |        0.3285       | 0.045957361437810526 |
|   3    |  0.3156666666666663 | 0.06499186837248325  |
|   4    |  0.3010000000000001 |  0.0788392797585694  |
|   5    |  0.2881999999999999 | 0.09193025569575682  |
|   6    |  0.2781666666666668 | 0.10368574283030749  |
|   7    | 0.26528571428571424 | 0.11308066816648561  |
|   8    |       0.25475       | 0.12146416860303043  |
|   9    | 0.24988888888888897 | 0.13262686270302135  |
|   10   | 0.24270000000000005 | 0.14088886689367552  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 3.4918197617743125

Per User RMSE (best)
+--------+--------------------+-------+
| userId |        rmse        | count |
+-----

recommendations finished on 1000/1000 queries. users per second: 1579.37


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.24700000000000016 | 0.013718717353605097 |
|   2    | 0.21800000000000003 | 0.022201854423126246 |
|   3    | 0.20433333333333334 | 0.030421814268367483 |
|   4    | 0.19775000000000004 | 0.038213625665178455 |
|   5    | 0.19060000000000007 | 0.045976399676383325 |
|   6    | 0.18599999999999994 | 0.054364329379747375 |
|   7    | 0.18128571428571422 | 0.06155757876630905  |
|   8    |       0.176875      | 0.06780341799846576  |
|   9    | 0.17155555555555552 | 0.07359106293967094  |
|   10   | 0.16820000000000002 |  0.078601598008863   |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.890729291379845

Per User RMSE (best)
+--------+----------------------+-------+
| userId |         rmse         | count |
+--

[{'precision_recall_by_user': Columns:
  	userId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 18000
  
  Data:
  +--------+--------+-----------+--------+-------+
  | userId | cutoff | precision | recall | count |
  +--------+--------+-----------+--------+-------+
  | 29099  |   1    |    0.0    |  0.0   |   30  |
  | 29099  |   2    |    0.0    |  0.0   |   30  |
  | 29099  |   3    |    0.0    |  0.0   |   30  |
  | 29099  |   4    |    0.0    |  0.0   |   30  |
  | 29099  |   5    |    0.0    |  0.0   |   30  |
  | 29099  |   6    |    0.0    |  0.0   |   30  |
  | 29099  |   7    |    0.0    |  0.0   |   30  |
  | 29099  |   8    |    0.0    |  0.0   |   30  |
  | 29099  |   9    |    0.0    |  0.0   |   30  |
  | 29099  |   10   |    0.0    |  0.0   |   30  |
  +--------+--------+-----------+--------+-------+
  [18000 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more ro